# Practical Exam: Spectrum Shades LLC
Spectrum Shades LLC is a prominent supplier of concrete color solutions, offering a wide range of pigments and coloring systems used in various concrete applications, including decorative concrete, precast concrete, and concrete pavers. The company prides itself on delivering high-quality colorants that meet the unique needs of its diverse clientele, including contractors, architects, and construction companies.
</br></br>
The company has recently observed a growing number of customer complaints regarding inconsistent color quality in their products. The discrepancies have led to a decline in customer satisfaction and a potential increase in product returns.
By identifying and mitigating the factors causing color variations, the company can enhance product reliability, reduce customer complaints, and minimize return rates.
</br></br>
You are part of the data analysis team tasked with providing actionable insights to help Spectrum Shades LLC address the issues of inconsistent color quality and improve customer satisfaction.

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `production_data` table, and the data team have provided the following data description. 

Write a query to ensure the data matches the description provided, including identifying and cleaning all invalid values. You must match all column names and description criteria.
</br>

- You should start with the data in the file "production_data.csv".
- Your output should be a DataFrame named clean_data.
- All column names and values should match the table below.
</br>

| Column Name             | Criteria                                                                                         |
|--------------------------|--------------------------------------------------------------------------------------------------|
| batch_id | Discrete. Identifier for each batch. Missing values are not possible. |
| production_date | Date. Date when the batch was produced.|
| raw_material_supplier | Categorical. Supplier of the raw materials. (1='national_supplier', 2='international_supplier'). <br> Missing values should be replaced with 'national_supplier'.|
| pigment_type           | Nominal. Type of pigment used. ['type_a', 'type_b', 'type_c']. <br> Missing values should be replaced with 'other'. |
| pigment_quantity       | Continuous. Amount of pigment added (in kilograms) (Range: 1 - 100). <br> Missing values should be replaced with median. |
| mixing_time           | Continuous. Duration of the mixing process (in minutes). <br> Missing values should be replaced with mean. |
| mixing_speed          | Categorical. Speed of the mixing process represented as categories: 'Low', 'Medium', 'High'.</br> Missing values should be replaced with 'Not Specified'. |
| product_quality_score | Continuous. Overall quality score of the final product (rating on a scale of 1 to 10). <br> Missing values should be replaced with mean. |


In [1]:
# Write your answer to Task 1 here
import pandas as pd
import numpy as np

# Load the data from the CSV file
df = pd.read_csv("production_data.csv")

# Replace common representations of missing values (e.g., '-', 'missing') with NaN
df.replace({'-': np.nan, 'missing': np.nan}, inplace=True)

# Rename columns if needed to match the description
df.rename(columns={
    "batch_id": "batch_id",
    "production_date": "production_date",
    "raw_material_supplier": "raw_material_supplier",
    "pigment_type": "pigment_type",
    "pigment_quantity": "pigment_quantity",
    "mixing_time": "mixing_time",
    "mixing_speed": "mixing_speed",
    "product_quality_score": "product_quality_score"
}, inplace=True)

# Clean batch_id (No missing values allowed)
df.dropna(subset=['batch_id'], inplace=True)

# Clean production_date (Ensure it's a date)
df['production_date'] = pd.to_datetime(df['production_date'], errors='coerce')
df.dropna(subset=['production_date'], inplace=True)

# Clean raw_material_supplier (Replace missing with 'national_supplier' and map values)
df['raw_material_supplier'].fillna('national_supplier', inplace=True)
df['raw_material_supplier'] = df['raw_material_supplier'].map({1: 'national_supplier', 2: 'international_supplier'})

# Ensure no invalid values in 'raw_material_supplier'
df = df[df['raw_material_supplier'].isin(['national_supplier', 'international_supplier'])]

# Clean pigment_type (Replace missing with 'other' and ensure only valid values)
df['pigment_type'].fillna('other', inplace=True)
valid_pigment_types = ['type_a', 'type_b', 'type_c', 'other']
df['pigment_type'] = df['pigment_type'].str.lower()  # Normalize case
df = df[df['pigment_type'].isin(valid_pigment_types)]

# Clean pigment_quantity (Replace missing with median and ensure range 1-100)
df['pigment_quantity'].fillna(df['pigment_quantity'].median(), inplace=True)
df = df[(df['pigment_quantity'] >= 1) & (df['pigment_quantity'] <= 100)]

# Clean mixing_time (Replace missing with mean)
df['mixing_time'].fillna(df['mixing_time'].mean(), inplace=True)

# Clean mixing_speed (Replace missing with 'Not Specified' and ensure only valid values)
df['mixing_speed'].fillna('Not Specified', inplace=True)
valid_mixing_speeds = ['low', 'medium', 'high', 'not specified']
df['mixing_speed'] = df['mixing_speed'].str.lower()  # Normalize case
df = df[df['mixing_speed'].isin(valid_mixing_speeds)]

# Clean product_quality_score (Replace missing with mean and ensure range 1-10)
df['product_quality_score'].fillna(df['product_quality_score'].mean(), inplace=True)
df = df[(df['product_quality_score'] >= 1) & (df['product_quality_score'] <= 10)]

# Assign the cleaned DataFrame to clean_data
clean_data = df.copy()

# Display the cleaned DataFrame without the index
print(clean_data.to_string(index=False))


 batch_id production_date  raw_material_supplier pigment_type  pigment_quantity  mixing_time  mixing_speed  product_quality_score
        1      2024-06-25      national_supplier       type_a         42.822881    22.560000          high               7.165102
        2      2023-11-23 international_supplier       type_b         42.873479    44.970000          high               6.849126
        3      2024-02-18 international_supplier       type_b         33.251206    52.620000          high               5.661209
        4      2023-11-11      national_supplier       type_a         52.532817    24.960000          high               6.991735
        5      2024-04-11 international_supplier       type_c         36.205108    25.370000          high               7.095043
        6      2024-02-14 international_supplier       type_b         29.991903    45.960000        medium               6.136382
        7      2023-08-25 international_supplier       type_a         35.941439    32.1300

# Task 2

You want to understand how the supplier type and quantity of materials affect the final product attributes.

Calculate the average `product_quality_score` and `pigment_quantity` grouped by `raw_material_supplier`.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named aggregated_data.
- It should include the three columns: `raw_material_supplier`, `avg_product_quality_score`, and `avg_pigment_quantity`.
- Your answers should be rounded to 2 decimal places.


In [2]:
# Write your answer to Task 2 here
import pandas as pd

# Load the data from the CSV file
df = pd.read_csv("production_data.csv")

# Calculate the average product_quality_score and pigment_quantity grouped by raw_material_supplier
aggregated_data = df.groupby('raw_material_supplier').agg(
    avg_product_quality_score=('product_quality_score', 'mean'),
    avg_pigment_quantity=('pigment_quantity', 'mean')
).round(2).reset_index()

# Display the aggregated data without the index
print(aggregated_data.to_string(index=False))

 raw_material_supplier  avg_product_quality_score  avg_pigment_quantity
                     1                       8.02                 44.73
                     2                       5.97                 34.91


# Task 3

To get more insight into the factors behind product quality, you want to filter the data to see an average product quality score for a specified set of results.

Identify the average `product_quality_score` for batches with a `raw_material_supplier` of 2 and a `pigment_quantity` greater than 35 kg.

Write a query to return the average `avg_product_quality_score` for these filtered batches. Use the original production data table, not the output of Task 2.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named pigment_data.
- It should consist of a 1-row DataFrame with 3 columns: `raw_material_supplier`, `pigment_quantity`, and `avg_product_quality_score`.
- Your answers should be rounded to 2 decimal places where appropriate.


In [3]:
# Write your answer to Task 3 here
# Load the data from the CSV file
df = pd.read_csv("production_data.csv")

# Filter the dataframe
filtered_df = df[(df['raw_material_supplier'] == 2) & (df['pigment_quantity'] > 35)]

# Calculate the average product quality score
avg_product_quality_score = filtered_df['product_quality_score'].mean()
pigment_quanity = filtered_df['pigment_quantity'].mean()

# Main outcome
pigment_data = pd.DataFrame({
    'raw_material_supplier': [2],
    'pigment_quantity': [round(pigment_quanity, 2)],
    'avg_product_quality_score': [round(avg_product_quality_score, 2)]
    })

print(pigment_data.to_string(index=False))

 raw_material_supplier  pigment_quantity  avg_product_quality_score
                     2             39.01                       5.97


# Task 4

In order to proceed with further analysis later, you need to analyze how various factors relate to product quality. Start by calculating the mean and standard deviation for the following columns: `pigment_quantity`, and `product_quality_score`. </br> These statistics will help in understanding the central tendency and variability of the data related to product quality.
</br> </br >
Next, calculate the Pearson correlation coefficient between the following variables: `pigment_quantity`, and `product_quality_score`.
</br>
These correlation coefficients will provide insights into the strength and direction of the relationships between the factors and overall product quality.


- You should start with the data in the file 'production_data.csv'.
- Calculate the mean and standard deviation for the columns pigment_quantity and product_quality_score as: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`.
- Calculate the Pearson correlation coefficient between pigment_quantity and product_quality_score as: `corr_coef`
- Your output should be a DataFrame named product_quality.
- It should include the columns: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`, `corr_coef`.
- Ensure that your answers are rounded to 2 decimal places.


In [4]:
# Write your answer to Task 4 here

# Load the data from the CSV file
df = pd.read_csv("production_data.csv")

# Calculate mean and standard deviation for 'pigment_quantity'
pigment_quantity_mean = round(df['pigment_quantity'].mean(), 2)
pigment_quantity_sd = round(df['pigment_quantity'].std(), 2)

# Calculate mean and standard deviation for 'product_quality_score'
product_quality_score_mean = round(df['product_quality_score'].mean(), 2)
product_quality_score_sd = round(df['product_quality_score'].std(), 2)

# Calculate Pearson correlation coefficient
corr_coef = round(df['pigment_quantity'].corr(df['product_quality_score'], method='pearson'), 2)

# Create the output DataFrame
product_quality = pd.DataFrame({
    'product_quality_score_mean': [product_quality_score_mean],
    'product_quality_score_sd': [product_quality_score_sd],
    'pigment_quantity_mean': [pigment_quantity_mean],
    'pigment_quantity_sd': [pigment_quantity_sd],
    'corr_coef': [corr_coef]
})

# Display the result without the index
print(product_quality.to_string(index=False))

 product_quality_score_mean  product_quality_score_sd  pigment_quantity_mean  pigment_quantity_sd  corr_coef
                       6.68                      1.39                  38.35                 6.83       0.49
